In [ ]:
import tqdm
import time
import openai
import regex as re
import textwrap
import pandas as pd
openai.api_key = ""

In [ ]:
def post_process(text):
    text = text.strip()
    text = text.lower()
    text = text.replace('\n', ' ')
    text = text.replace(" .", ".")
    text = text.replace(" ?", "?")
    text = text.replace("/.", "")
    text = text.replace("/", "")
    text = text.replace(" ,", ",")
    text = text.replace("//.", "")
    text = re.sub(r' +', ' ', text)
    text = text.replace('\\', '')
    return text

In [ ]:
ds = pd.read_csv("cookie.csv")
df = pd.DataFrame(columns=['text', 'label'], index=range(len(ds)))
wrapper = textwrap.TextWrapper(width=100)
for i in range(len(ds)):
    idx = ds.iat[i, 0]
    text = ds.iat[i, 1]
    label = ds.iat[i, 2]
    text = post_process(text)
    text = wrapper.wrap(text=text)
    textw = "\n".join(c for c in text)
    df.loc[i] = [textw, label]

split = 5
df_train = df[:split]
df_val = df[split:]

id_to_label = {0:'diseased subject', 1:'healthy subject'}
train_instructions = [f"Transcript: {x}\nAssistant: {id_to_label[y]}" for x, y in zip(df_train.text, df_train.label)]
val_instructions = [f"Transcript: {x}\nAssistant:" for x in df_val.text]
val_labels = [id_to_label[y] for y in df_val.label]


In [ ]:
template = """You are helpful assistant, who is given a transcript of
a conversation from either from a healthy or diseased subject.
Please classify it as one of them and if context is not enough
classify as diseased subject. A few examples are given below:\n"""

prompt = f"{template}\n"
for ex in train_instructions:
    prompt += f"{ex}\n\n"
print(prompt)

You are helpful assistant, who is given a transcript of 
a conversation from either from a healthy or diseased subject. 
Please classify it as one of them and if context is not enough
classify as diseased subject. A few examples are given below:

Transcript: well â€¡ the stool is falling over. the boy is taking cookies out of the cookie jar. the little girl
is reaching for a cookie and holding her arm up. the mother is doing dishes. the water is spilling
out of the sink. and the mothers standing in the water. the mother is looking out of the window. the
waters coming out of the faucet. that is about all i can get out of that.
Assistant: diseased subject

Transcript: these two little kids are getting cookies out of the cookie jar. and this lady over here is ... what
is she doing? she is decorating. is not that what she is doing? decorating? well, they are getting
cookies out of the cookie jar. up on a stool that looks like it is gonna topple over. looks like she
is washing dishes. dryin

In [ ]:
responses, labels = [], []
for i in range(10):
    for idx in tqdm.tqdm(range(i*50, (i+1)*50)):
        try:
            completion = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo-0613",
                        messages=[{"role": "user", "content": prompt + f"{val_instructions[idx]}\n"}],
                        temperature=0,
                        request_timeout=20,
                    )
            response = completion.choices[0].message.content
        except:
            continue
        label = val_labels[idx]
        labels.append(label)
        responses.append(response)
    if i < 9:
        time.sleep(10)


100%|██████████| 50/50 [01:05<00:00,  1.31s/it]


In [ ]:
# with open("labels.txt", "r") as f:
#     labels = f.readlines()
# with open("responses.txt", "r") as f:
#     responses = f.readlines()

results_idx = [0 if "diseased subject" in x else 1 for x in responses]
labels_idx =  [0 if "diseased subject" in x else 1 for x in labels]
assert len(results_idx) == len(labels_idx)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
tn, fp, fn, tp = confusion_matrix(results_idx, labels_idx).ravel()
specificity = tn / (tn+fp)

print("Accuracy:           {:.4f}".format(accuracy_score(results_idx, labels_idx)))
print("F1 Score:           {:.4f}".format(f1_score(results_idx, labels_idx)))
print("Precision:          {:.4f}".format(precision_score(results_idx, labels_idx)))
print("Specificity:        {:.4f}".format(specificity))
print("Recall/Sensitivity: {:.4f}".format(recall_score(results_idx, labels_idx)))

Accuracy:           0.5046
F1 Score:           0.6485
Precision:          0.8143
Specificity:        0.3133
Recall/Sensitivity: 0.5388
